In [ ]:
"""
This notebook is for cleaning and preprocessing pond data
Two labels will be added:
    1-water safety score
    2-Fish health score
    These scores will be defined based on outliers identified in the data
after all preprocessing all csvs will be concatenated and data shall be balanced for classification 
Different classification models shall be tested
LSTM/RNN/GRU can be used for prediction because we are dealing with time series data
"""
"""
we compute the weight based on: 
High variability (coefficient of variation)
High outlier frequency (many values outside IQR bounds)
"""

"""
Understand the code
calculate fish health score
concatenate dataframes
"""
"""
https://globalflyfisher.com/fish-better/fultons-condition-factor-calculator
"""

In [33]:
#pond data for water condition
import pandas as pd
import numpy as np

pond1=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond1.csv")
pond2=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond2.csv")
pond3=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond3.csv")
pond4=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond4.csv")
pond6=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond6.csv")
pond7=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTPond7.csv")
pond8=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond8.csv")
pond9=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond9.csv")
pond10=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond10.csv")
pond11=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond11.csv")
pond12=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond12.csv")

pond1.head()

C:\Users\kaout\AppData\Local\Temp\ipykernel_30176\215217444.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  pond4=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond4.csv")
C:\Users\kaout\AppData\Local\Temp\ipykernel_30176\215217444.py:10: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  pond7=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTPond7.csv")
C:\Users\kaout\AppData\Local\Temp\ipykernel_30176\215217444.py:11: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  pond8=pd.read_csv("Sensor_based_aquaponics_fish_pond_data/IoTpond8.csv")


,created_at,entry_id,Temperature (C),Turbidity(NTU),Dissolved Oxygen(g/ml),PH,Ammonia(g/ml),Nitrate(g/ml),Population,Fish_Length(cm),Fish_Weight(g)
0,2021-06-19 00:00:05 CET,1889,24.8750,100,4.505,8.43365,0.45842,193,50,7.11,2.91
1,2021-06-19 00:01:02 CET,1890,24.9375,100,6.601,8.43818,0.45842,194,50,7.11,2.91
2,2021-06-19 00:01:22 CET,1891,24.8750,100,15.797,8.42457,0.45842,192,50,7.11,2.91
3,2021-06-19 00:01:44 CET,1892,24.9375,100,5.046,8.43365,0.45842,193,50,7.11,2.91
4,2021-06-19 00:02:07 CET,1893,24.9375,100,38.407,8.40641,0.45842,192,50,7.11,2.91


In [61]:
print("Data columns:")
print(pond12.columns)
print("\nData info:")
print(pond12.info())
print("\nData shape:")
print(pond12.shape)

Data columns:
Index(['created_at', 'entry_id', 'TEMPERATURE', 'TURBIDITY', 'DISOLVED OXYGEN',
       'pH', 'AMMONIA', 'NITRATE', 'Population', 'Length', 'Weight'],
      dtype='object')

Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       3590 non-null   object 
 1   entry_id         3590 non-null   int64  
 2   TEMPERATURE      3590 non-null   float64
 3   TURBIDITY        3590 non-null   int64  
 4   DISOLVED OXYGEN  3590 non-null   float64
 5   pH               3590 non-null   float64
 6   AMMONIA          3590 non-null   float64
 7   NITRATE          3590 non-null   int64  
 8   Population       3590 non-null   int64  
 9   Length           3590 non-null   float64
 10  Weight           3590 non-null   float64
dtypes: float64(6), int64(4), object(1)
memory usage: 308.6+ KB
None

Data shape:
(3590, 11)


In [35]:
#Pond7 contains additional empty columns to drop
pond7=pond7.drop(columns=["Unnamed: 11","Unnamed: 12","Unnamed: 13"])

In [65]:
pond3=pond3.dropna()
pond3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169078 entries, 0 to 169184
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   created_at              169078 non-null  object 
 1   entry_id                169078 non-null  int64  
 2   Temperature(C)          169078 non-null  float64
 3   Turbidity(NTU)          169078 non-null  int64  
 4   Dissolved Oxygen(g/ml)  169078 non-null  float64
 5   PH                      169078 non-null  float64
 6   Ammonia(g/ml)           169078 non-null  float64
 7   Nitrate(g/ml)           169078 non-null  int64  
 8   Population              169078 non-null  int64  
 9   Fish_Length(cm)         169078 non-null  float64
 10  Fish_Weight(g)          169078 non-null  float64
dtypes: float64(6), int64(4), object(1)
memory usage: 15.5+ MB


In [67]:
import pandas as pd
import numpy as np

def calculate_iqr_ranges(df, param_name):
    data = df[param_name].dropna()
    # Calculate quartiles and IQR
    Q1 = data.quantile(0.25)
    Q2 = data.quantile(0.50)  # Median
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate outlier boundaries
    lower_outlier = Q1 - 1.5 * IQR
    upper_outlier = Q3 + 1.5 * IQR
    
    # Define ranges
    ranges = {
        'optimal': (Q1, Q3),              # Middle 50% (IQR itself)
        'acceptable': (Q1 - 0.5*IQR, Q3 + 0.5*IQR),  # Slightly extended
        'critical_low': lower_outlier,     # Below this = outlier
        'critical_high': upper_outlier,    # Above this = outlier
        'Q1': Q1,
        'Q2': Q2,
        'Q3': Q3,
        'IQR': IQR
    }
    return ranges


def calculate_parameter_weights(df, param_list):
    weights = {}
    
    for param in param_list:
        data = df[param].dropna()
        if len(data) < 3:
            weights[param] = 0
            print(f"  {param}: Insufficient data, setting weight to 0")
            continue
        
        mean = data.mean()
        std = data.std()
        
        if pd.isna(mean) or pd.isna(std):
            cv = 0
        elif abs(mean) < 1e-10:  # Mean too close to zero
            cv = std  # Use std directly instead of CV
        else:
            cv = abs(std / mean)  # Use absolute value to handle negative means
        
        # Replace inf or nan with 0
        if not np.isfinite(cv):
            cv = 0
        
        # Calculate outlier percentage
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        
        if IQR == 0:
            outlier_pct = 0
        else:
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outlier_pct = ((data < lower_bound).sum() + (data > upper_bound).sum()) / len(data)
        
        # Combined score: 60% CV + 40% outlier frequency
        combined_score = 0.6 * cv + 0.4 * outlier_pct
        
        # Final safety check
        if not np.isfinite(combined_score):
            combined_score = 0
        
        weights[param] = combined_score
    
    # Normalize weights to sum to 1.0
    total_weight = sum(weights.values())
    
    if total_weight == 0 or not np.isfinite(total_weight):
        weights = {k: 1.0/len(param_list) for k in param_list}
    else:
        weights = {k: v/total_weight for k, v in weights.items()}
    
    return weights


def build_safety_ranges_from_data(df, manual_overrides=None):
    # Define parameters to analyze
    param_list = [
        'Temperature(C)',
        'Dissolved Oxygen(g/ml)',
        'PH',
        'Ammonia(g/ml)',
        'Turbidity(NTU)',
        'Nitrate(g/ml)'
    ]
    # Filter to existing columns
    param_list = [p for p in param_list if p in df.columns]
    
    if not param_list:
        print("ERROR: No valid water quality parameters found in dataframe!")
        print(f"Available columns: {df.columns.tolist()}")
        return {}
    
    print("\n" + "="*70)
    print("DATA-DRIVEN SAFETY RANGES (IQR METHOD)")
    print("="*70)
    print(f"Parameters found: {len(param_list)}")
    print(f"Total rows: {len(df)}")
    
    # Calculate weights
    weights = calculate_parameter_weights(df, param_list)
    
    # Build ranges dictionary
    SAFETY_RANGES = {}
    
    for param in param_list:
        try:
            # Check if parameter has enough valid data
            valid_data = df[param].dropna()
            if len(valid_data) < 3:
                print(f"\n {param}: Insufficient data ({len(valid_data)} values), skipping...")
                continue
            
            ranges = calculate_iqr_ranges(df, param)
            
            # Validate ranges (check for NaN or infinite values)
            if not all(np.isfinite([ranges['optimal'][0], ranges['optimal'][1], 
                                   ranges['acceptable'][0], ranges['acceptable'][1],
                                   ranges['critical_low'], ranges['critical_high']])):
                print(f"\n {param}: Invalid ranges detected, skipping...")
                continue
            
            # Apply manual overrides if provided
            if manual_overrides and param in manual_overrides:
                print(f"\n{param}: Applying manual overrides")
                ranges.update(manual_overrides[param])
            
            SAFETY_RANGES[param] = {
                'optimal': ranges['optimal'],
                'acceptable': ranges['acceptable'],
                'critical_low': ranges['critical_low'],
                'critical_high': ranges['critical_high'],
                'weight': weights[param]
            }
            
            # Print summary
            print(f"\n✓ {param}:")
            print(f"  Data points: {len(valid_data)}")
            print(f"  Optimal:    {ranges['optimal'][0]:.3f} - {ranges['optimal'][1]:.3f}")
            print(f"  Acceptable: {ranges['acceptable'][0]:.3f} - {ranges['acceptable'][1]:.3f}")
            print(f"  Critical:   < {ranges['critical_low']:.3f} or > {ranges['critical_high']:.3f}")
            print(f"  Weight:     {weights[param]:.4f} ({weights[param]*100:.2f}%)")
            
        except Exception as e:
            print(f"\n ERROR processing {param}: {str(e)}")
            continue
    
    print("\n" + "="*70)
    print(f"Successfully generated ranges for {len(SAFETY_RANGES)} parameters")
    print("="*70)
    
    if not SAFETY_RANGES:
        print("\n WARNING: No safety ranges were generated!")
        print("Possible reasons:")
        print("  - Insufficient data in all columns")
        print("  - All values are NaN")
        print("  - Column names don't match expected parameter names")
    
    return SAFETY_RANGES


def export_ranges_to_file(SAFETY_RANGES, filename='safety_ranges.py'):
    """
    Export ranges as a Python file
    """
    with open(filename, 'w') as f:
        f.write("# Auto-generated safety ranges using IQR method\n\n")
        f.write("SAFETY_RANGES = {\n")
        
        for param, ranges in SAFETY_RANGES.items():
            f.write(f"    '{param}': {{\n")
            f.write(f"        'optimal': ({ranges['optimal'][0]:.4f}, {ranges['optimal'][1]:.4f}),\n")
            f.write(f"        'acceptable': ({ranges['acceptable'][0]:.4f}, {ranges['acceptable'][1]:.4f}),\n")
            f.write(f"        'critical_low': {ranges['critical_low']:.4f},\n")
            f.write(f"        'critical_high': {ranges['critical_high']:.4f},\n")
            f.write(f"        'weight': {ranges['weight']:.4f}\n")
            f.write(f"    }},\n")
        
        f.write("}\n")
    
    print(f"\n✓ Safety ranges exported to: {filename}")


if __name__ == "__main__":
    df = pond3.copy()
    print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    SAFETY_RANGES = build_safety_ranges_from_data(df) 

Dataset loaded: 169078 rows, 11 columns

DATA-DRIVEN SAFETY RANGES (IQR METHOD)
Parameters found: 6
Total rows: 169078

✓ Temperature(C):
  Data points: 169078
  Optimal:    23.375 - 24.125
  Acceptable: 23.000 - 24.500
  Critical:   < 22.250 or > 25.250
  Weight:     0.0327 (3.27%)

✓ Dissolved Oxygen(g/ml):
  Data points: 169078
  Optimal:    5.080 - 8.088
  Acceptable: 3.576 - 9.592
  Critical:   < 0.568 or > 12.600
  Weight:     0.3885 (38.85%)

✓ PH:
  Data points: 169078
  Optimal:    7.244 - 7.344
  Acceptable: 7.194 - 7.394
  Critical:   < 7.095 or > 7.494
  Weight:     0.0805 (8.05%)

✓ Ammonia(g/ml):
  Data points: 169078
  Optimal:    5.759 - 1009.766
  Acceptable: -496.245 - 1511.770
  Critical:   < -1500.252 or > 2515.777
  Weight:     0.0616 (6.16%)

✓ Turbidity(NTU):
  Data points: 169078
  Optimal:    80.000 - 100.000
  Acceptable: 70.000 - 110.000
  Critical:   < 50.000 or > 130.000
  Weight:     0.1689 (16.89%)

✓ Nitrate(g/ml):
  Data points: 169078
  Optimal:    123

C:\Users\kaout\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [68]:
import pandas as pd
import numpy as np

def calculate_parameter_score(value, param_name):
    if param_name not in SAFETY_RANGES:
        return np.nan
    
    ranges = SAFETY_RANGES[param_name]
    opt_low, opt_high = ranges['optimal']
    acc_low, acc_high = ranges['acceptable']
    crit_low = ranges['critical_low']
    crit_high = ranges['critical_high']
    
    # Handle NaN values
    if pd.isna(value):
        return 0
    # OPTIMAL RANGE = 100 points
    if opt_low <= value <= opt_high:
        return 100
    
    # ACCEPTABLE RANGE = 75-99 points (linear interpolation)
    elif acc_low <= value < opt_low:
        # Between acceptable_low and optimal_low
        score = 75 + 25 * (value - acc_low) / (opt_low - acc_low)
        return score
    
    elif opt_high < value <= acc_high:
        # Between optimal_high and acceptable_high
        score = 75 + 25 * (acc_high - value) / (acc_high - opt_high)
        return score
    
    # WARNING ZONE = 25-74 points
    elif crit_low < value < acc_low:
        # Between critical_low and acceptable_low
        score = 25 + 50 * (value - crit_low) / (acc_low - crit_low)
        return score
    
    elif acc_high < value < crit_high:
        # Between acceptable_high and critical_high
        score = 25 + 50 * (crit_high - value) / (crit_high - acc_high)
        return score
    
    # CRITICAL ZONE = 0-24 points
    else:
        # Beyond critical thresholds
        if value <= crit_low:
            # Extremely low
            return max(0, 24 * (value / crit_low)) if crit_low > 0 else 0
        else:
            # Extremely high
            excess_ratio = (value - crit_high) / crit_high if crit_high > 0 else 1
            return max(0, 24 * (1 / (1 + excess_ratio)))


def add_safety_scores(df):
    # Calculate individual parameter scores
    for param in SAFETY_RANGES.keys():
        if param in df.columns:
            score_col = f'{param}_Safety_Score'
            df[score_col] = df[param].apply(lambda x: calculate_parameter_score(x, param))
            print(f"✓ Calculated: {score_col}")
    
    # Calculate overall weighted safety score
    df['Water_Safety_Score'] = 0
    total_weight = 0
    
    for param, ranges in SAFETY_RANGES.items():
        if param in df.columns:
            score_col = f'{param}_Safety_Score'
            weight = ranges['weight']
            df['Water_Safety_Score'] += df[score_col] * weight
            total_weight += weight
    
    # Normalize if weights don't sum to 1.0
    if total_weight != 1.0:
        df['Water_Safety_Score'] = df['Water_Safety_Score'] / total_weight
        
    # Add safety category
    def get_safety_category(score):
        if score >= 75:
            return 'SAFE'
        elif score >= 50:
            return 'WARNING'
        elif score >= 25:
            return 'DANGER'
        else:
            return 'CRITICAL'
    
    df['Safety_Category'] = df['Water_Safety_Score'].apply(get_safety_category)
    return df

if __name__ == "__main__":
    df = pond3.copy()    
    df = add_safety_scores(df)
    output_file = 'pond3_with_safety_scores.csv'
    df.to_csv(output_file, index=False)

✓ Calculated: Temperature(C)_Safety_Score
✓ Calculated: Dissolved Oxygen(g/ml)_Safety_Score
✓ Calculated: PH_Safety_Score
✓ Calculated: Ammonia(g/ml)_Safety_Score
✓ Calculated: Turbidity(NTU)_Safety_Score
✓ Calculated: Nitrate(g/ml)_Safety_Score


In [69]:
data=pd.read_csv("pond3_with_safety_scores.csv")
data["Safety_Category"].value_counts()

Safety_Category
SAFE       119372
WARNING     46388
DANGER       3318
Name: count, dtype: int64

In [20]:
#concatenating dataframes
import pandas as pd
df1=pd.read_csv("pond1_with_safety_scores.csv")
df2=pd.read_csv("pond2_with_safety_scores.csv")
df3=pd.read_csv("pond3_with_safety_scores.csv")
df4=pd.read_csv("pond4_with_safety_scores.csv")
df6=pd.read_csv("pond6_with_safety_scores.csv")
df7=pd.read_csv("pond7_with_safety_scores.csv")
df8=pd.read_csv("pond8_with_safety_scores.csv")
df9=pd.read_csv("pond9_with_safety_scores.csv")
df10=pd.read_csv("pond10_with_safety_scores.csv")
df11=pd.read_csv("pond11_with_safety_scores.csv")
df12=pd.read_csv("pond12_with_safety_scores.csv")

C:\Users\kaout\AppData\Local\Temp\ipykernel_40140\2991685805.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df4=pd.read_csv("pond4_with_safety_scores.csv")
C:\Users\kaout\AppData\Local\Temp\ipykernel_40140\2991685805.py:8: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df7=pd.read_csv("pond7_with_safety_scores.csv")
C:\Users\kaout\AppData\Local\Temp\ipykernel_40140\2991685805.py:9: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df8=pd.read_csv("pond8_with_safety_scores.csv")


In [17]:
import pandas as pd
import numpy as np

def standardize_dataframe(df):
    # Create comprehensive column mapping
    column_mapping = {
        # Timestamp columns
        'created_at': 'timestamp',
        'Date': 'timestamp',
        'DATE': 'timestamp',
        
        # Entry ID
        'entry_id': 'entry_id',
        
        # Temperature variations
        'Temperature (C)': 'temperature',
        'Temperature(C)': 'temperature',
        'TEMPERATURE': 'temperature',
        
        # Turbidity variations
        'Turbidity(NTU)': 'turbidity',
        'Turbidity (NTU)': 'turbidity',
        'TURBIDITY': 'turbidity',
        
        # Dissolved Oxygen variations
        'Dissolved Oxygen(g/ml)': 'dissolved_oxygen',
        'Dissolved Oxygen (mg/L)': 'dissolved_oxygen',
        'DISOLVED OXYGEN': 'dissolved_oxygen',
        
        # pH variations
        'PH': 'ph',
        'pH': 'ph',
        
        # Ammonia variations
        'Ammonia(g/ml)': 'ammonia',
        'Ammonia (mg/L)': 'ammonia',
        'AMMONIA': 'ammonia',
        
        # Nitrate variations
        'Nitrate(g/ml)': 'nitrate',
        'Nitrate (mg/L)': 'nitrate',
        'NITRATE': 'nitrate',
        
        # Population
        'Population': 'population',
        
        # Fish Length variations
        'Fish_Length(cm)': 'fish_length',
        'Fish_Length (cm)': 'fish_length',
        'Fish_length(cm)': 'fish_length',
        'Total_length (cm)': 'fish_length',
        'Length': 'fish_length',
        'Lenght': 'fish_length',
        
        # Fish Weight variations
        'Fish_Weight(g)': 'fish_weight',
        'Fish_Weight (g)': 'fish_weight',
        'Fish_weight(g)': 'fish_weight',
        'Weight (g)': 'fish_weight',
        'Weight': 'fish_weight',
        
        # Safety scores to keep
        'Water_Safety_Score': 'water_safety_score',
        'Safety_Category': 'safety_category',
    }
    
    # Create new dataframe with standardized columns
    df_standardized = pd.DataFrame()
    
    # Map columns
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df_standardized[new_name] = df[old_name]
    
    # Define columns to keep in final output
    columns_to_keep = [
        'timestamp',
        'entry_id',
        'temperature',
        'turbidity',
        'dissolved_oxygen',
        'ph',
        'ammonia',
        'nitrate',
        'population',
        'fish_length',
        'fish_weight',
        'water_safety_score',
        'safety_category'
    ]
    
    # Keep only columns that exist
    final_columns = [col for col in columns_to_keep if col in df_standardized.columns]
    df_standardized = df_standardized[final_columns]
    
    return df_standardized


def concatenate_dataframes(df_list):
    print("\n" + "="*70)
    print("STANDARDIZING AND CONCATENATING DATAFRAMES")
    print("="*70)
    
    standardized_dfs = []
    
    for i, df in enumerate(df_list):
        print(f"\nProcessing DataFrame {i+1}:")
        print(f"  Original shape: {df.shape}")
        print(f"  Original columns: {len(df.columns)}")
        
        # Standardize
        df_std = standardize_dataframe(df)
        
        print(f"  Standardized shape: {df_std.shape}")
        print(f"  Standardized columns: {list(df_std.columns)}")
        df_std['source_df'] = i + 1
        standardized_dfs.append(df_std)
    
    df_combined = pd.concat(standardized_dfs, axis=0, ignore_index=True)
    
    print("\n" + "="*70)
    print("CONCATENATION COMPLETE")
    print("="*70)
    print(f"Combined shape: {df_combined.shape}")
    print(f"Total rows: {len(df_combined)}")
    print(f"Columns: {list(df_combined.columns)}")
    
    # Print summary statistics
    print("\nRows per source:")
    print(df_combined['source_df'].value_counts().sort_index())
    
    print("\nMissing values per column:")
    missing = df_combined.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        for col, count in missing.items():
            pct = (count / len(df_combined)) * 100
            print(f"  {col}: {count} ({pct:.2f}%)")
    else:
        print("  No missing values!")
    
    return df_combined

if __name__ == "__main__":
    all_dataframes = [df1,df2,df3,df4,df9,df10,df11,df12,df6]
    df_combined = concatenate_dataframes(all_dataframes)
    df_combined.to_csv('combined_data_part1_with_population.csv', index=False)


STANDARDIZING AND CONCATENATING DATAFRAMES

Processing DataFrame 1:
  Original shape: (83072, 19)
  Original columns: 19
  Standardized shape: (83072, 13)
  Standardized columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'population', 'fish_length', 'fish_weight', 'water_safety_score', 'safety_category']

Processing DataFrame 2:
  Original shape: (172249, 18)
  Original columns: 18
  Standardized shape: (172249, 13)
  Standardized columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'population', 'fish_length', 'fish_weight', 'water_safety_score', 'safety_category']

Processing DataFrame 3:
  Original shape: (169078, 19)
  Original columns: 19
  Standardized shape: (169078, 13)
  Standardized columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'population', 'fish_length', 'fish_weight', 'water_safety_score', 's

In [18]:
#dataframe with population

if __name__ == "__main__":
    all_dataframes = [df7,df8]
    df_combined2 = concatenate_dataframes(all_dataframes)
    df_combined2.to_csv('combined_data_part2_without_population.csv', index=False)



STANDARDIZING AND CONCATENATING DATAFRAMES

Processing DataFrame 1:
  Original shape: (70744, 15)
  Original columns: 15
  Standardized shape: (70744, 12)
  Standardized columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'fish_length', 'fish_weight', 'water_safety_score', 'safety_category']

Processing DataFrame 2:
  Original shape: (70744, 17)
  Original columns: 17
  Standardized shape: (70744, 12)
  Standardized columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'fish_length', 'fish_weight', 'water_safety_score', 'safety_category']

CONCATENATION COMPLETE
Combined shape: (141488, 13)
Total rows: 141488
Columns: ['timestamp', 'entry_id', 'temperature', 'turbidity', 'dissolved_oxygen', 'ph', 'ammonia', 'nitrate', 'fish_length', 'fish_weight', 'water_safety_score', 'safety_category', 'source_df']

Rows per source:
source_df
1    70744
2    70744
Name: count, dtype: i

In [19]:
print(df_combined.info())
print("\n")
print(df_combined2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764315 entries, 0 to 764314
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   timestamp           764315 non-null  object 
 1   entry_id            764315 non-null  float64
 2   temperature         764315 non-null  float64
 3   turbidity           764315 non-null  float64
 4   dissolved_oxygen    764315 non-null  float64
 5   ph                  764315 non-null  float64
 6   ammonia             764225 non-null  float64
 7   nitrate             764315 non-null  float64
 8   population          612530 non-null  float64
 9   fish_length         764315 non-null  object 
 10  fish_weight         764315 non-null  float64
 11  water_safety_score  764315 non-null  float64
 12  safety_category     764315 non-null  object 
 13  source_df           764315 non-null  int64  
dtypes: float64(10), int64(1), object(3)
memory usage: 81.6+ MB
None


<class 'pandas.cor

In [27]:
import pandas as pd
import numpy as np

def clean_fish_length(df):
    """Convert fish_length to numeric if it's stored as object"""
    if df['fish_length'].dtype == 'object':
        df['fish_length'] = pd.to_numeric(df['fish_length'], errors='coerce')
    return df


def calculate_fish_health_ranges_without_population(df):
    """
    Calculate health ranges using IQR method - NO population
    """
    df = clean_fish_length(df)
    
    # Remove rows where fish metrics are missing
    df_clean = df.dropna(subset=['fish_length', 'fish_weight'])
    
    print("\n" + "="*70)
    print("FISH HEALTH RANGES (WITHOUT POPULATION)")
    print("="*70)
    print(f"Valid records: {len(df_clean)} / {len(df)}")
    
    FISH_HEALTH_RANGES = {}
    
    # Fish Length (higher is better) - increased weight
    data = df_clean['fish_length']
    Q1, Q2, Q3 = data.quantile([0.25, 0.50, 0.75])
    IQR = Q3 - Q1
    FISH_HEALTH_RANGES['fish_length'] = {
        'optimal': (Q2, Q3),
        'acceptable': (Q1, Q3 + 0.5*IQR),
        'critical_low': Q1 - 1.5*IQR,
        'critical_high': Q3 + 1.5*IQR,
        'weight': 0.42,  # Increased from 0.35
        'direction': 'higher_better'
    }
    print(f"\nFish Length: {Q1:.2f} | {Q2:.2f} | {Q3:.2f}")
    
    # Fish Weight (higher is better) - increased weight
    data = df_clean['fish_weight']
    Q1, Q2, Q3 = data.quantile([0.25, 0.50, 0.75])
    IQR = Q3 - Q1
    FISH_HEALTH_RANGES['fish_weight'] = {
        'optimal': (Q2, Q3),
        'acceptable': (Q1, Q3 + 0.5*IQR),
        'critical_low': Q1 - 1.5*IQR,
        'critical_high': Q3 + 1.5*IQR,
        'weight': 0.42,  # Increased from 0.35
        'direction': 'higher_better'
    }
    print(f"Fish Weight: {Q1:.2f} | {Q2:.2f} | {Q3:.2f}")
    
    # Condition Factor weight - increased
    FISH_HEALTH_RANGES['condition_factor'] = {'weight': 0.16}  # Increased from 0.15
    
    print("\nWeights adjusted for missing population data:")
    print("  - Fish Length: 42%")
    print("  - Fish Weight: 42%")
    print("  - Condition Factor: 16%")
    
    return FISH_HEALTH_RANGES


def calculate_condition_factor(df):
    """Calculate Fulton's Condition Factor: K = (Weight / Length^3) * 100"""
    df = clean_fish_length(df)
    
    df['condition_factor'] = (df['fish_weight'] / (df['fish_length'] ** 3)) * 100
    df['condition_factor'] = df['condition_factor'].replace([np.inf, -np.inf], np.nan)
    
    return df


def calculate_metric_score(value, param_name, ranges):
    """Calculate health score (0-100) for a single metric"""
    if param_name not in ranges or pd.isna(value):
        return np.nan
    
    r = ranges[param_name]
    opt_low, opt_high = r['optimal']
    acc_low, acc_high = r['acceptable']
    crit_low = r['critical_low']
    crit_high = r['critical_high']
    direction = r.get('direction', 'higher_better')
    
    # Length/Weight: higher is better
    if opt_low <= value <= opt_high:
        return 100
    elif value > opt_high:
        return 100 if value <= acc_high else max(90, 100 - 10 * (value - acc_high) / (crit_high - acc_high))
    elif value < opt_low:
        if value >= acc_low:
            return 75 - 25 * (opt_low - value) / (opt_low - acc_low)
        elif value >= crit_low:
            return 25 + 50 * (value - crit_low) / (acc_low - crit_low)
        else:
            return 10


def score_condition_factor(k):
    """Score Condition Factor (K)"""
    if pd.isna(k):
        return np.nan
    if 1.0 <= k <= 1.5:
        return 100
    elif (0.8 <= k < 1.0) or (1.5 < k <= 2.0):
        return 75 + 25 * min((k - 0.8) / 0.2, (2.0 - k) / 0.5) if k < 1.0 else 75 + 25 * (2.0 - k) / 0.5
    elif 0.5 <= k < 0.8 or 2.0 < k <= 3.0:
        return 25 + 50 * min((k - 0.5) / 0.3, (3.0 - k) / 1.0) if k < 0.8 else 25 + 50 * (3.0 - k) / 1.0
    else:
        return 10


def add_fish_health_scores_without_population(df):
    """
    Add fish health scores to dataframe (NO population)
    """
    print("\n" + "="*70)
    print("CALCULATING FISH HEALTH SCORES (WITHOUT POPULATION)")
    print("="*70)
    
    # Clean and calculate condition factor
    df = clean_fish_length(df)
    df = calculate_condition_factor(df)
    
    # Calculate ranges
    FISH_HEALTH_RANGES = calculate_fish_health_ranges_without_population(df)
    
    # Individual metric scores
    df['fish_length_score'] = df['fish_length'].apply(
        lambda x: calculate_metric_score(x, 'fish_length', FISH_HEALTH_RANGES)
    )
    df['fish_weight_score'] = df['fish_weight'].apply(
        lambda x: calculate_metric_score(x, 'fish_weight', FISH_HEALTH_RANGES)
    )
    df['condition_factor_score'] = df['condition_factor'].apply(score_condition_factor)
    
    # Overall Fish Health Score (adjusted weights without population)
    df['fish_health_score'] = (
        0.42 * df['fish_length_score'].fillna(0) +
        0.42 * df['fish_weight_score'].fillna(0) +
        0.16 * df['condition_factor_score'].fillna(0)
    )
    
    # Health Category
    def get_health_category(score):
        if pd.isna(score):
            return 'UNKNOWN'
        if score >= 80:
            return 'EXCELLENT'
        elif score >= 65:
            return 'GOOD'
        elif score >= 50:
            return 'FAIR'
        elif score >= 35:
            return 'POOR'
        else:
            return 'CRITICAL'
    
    df['fish_health_category'] = df['fish_health_score'].apply(get_health_category)
    
    return df


if __name__ == "__main__":
    
    df = df_combined2.copy()    
    df = add_fish_health_scores_without_population(df)
    df.to_csv('data_with_fish_health_no_population.csv', index=False)


CALCULATING FISH HEALTH SCORES (WITHOUT POPULATION)

FISH HEALTH RANGES (WITHOUT POPULATION)
Valid records: 141488 / 141488

Fish Length: 16.87 | 18.04 | 32.09
Fish Weight: 50.53 | 61.11 | 300.20

Weights adjusted for missing population data:
  - Fish Length: 42%
  - Fish Weight: 42%
  - Condition Factor: 16%


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141488 entries, 0 to 141487
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   timestamp               141488 non-null  object 
 1   entry_id                141488 non-null  int64  
 2   temperature             141488 non-null  float64
 3   turbidity               141488 non-null  int64  
 4   dissolved_oxygen        141488 non-null  float64
 5   ph                      141488 non-null  float64
 6   ammonia                 141488 non-null  float64
 7   nitrate                 141488 non-null  int64  
 8   fish_length             141488 non-null  float64
 9   fish_weight             141488 non-null  float64
 10  water_safety_score      141488 non-null  float64
 11  safety_category         141488 non-null  object 
 12  source_df               141488 non-null  int64  
 13  condition_factor        141488 non-null  float64
 14  fish_length_score   

In [25]:
def clean_fish_length(df):
    """Convert fish_length to numeric if it's stored as object"""
    if df['fish_length'].dtype == 'object':
        df['fish_length'] = pd.to_numeric(df['fish_length'], errors='coerce')
    return df


def calculate_condition_factor(df):
    """
    Calculate Fulton's Condition Factor (K-factor)
    K = (Weight / Length³) × 100
    
    Healthy range: 1.0 - 1.5
    """
    df = clean_fish_length(df)
    
    df['condition_factor'] = (df['fish_weight'] / (df['fish_length'] ** 3)) * 100
    df['condition_factor'] = df['condition_factor'].replace([np.inf, -np.inf], np.nan)
    
    print(f"\n✓ Condition Factor (K) calculated")
    print(f"  Mean K: {df['condition_factor'].mean():.3f}")
    print(f"  Min K: {df['condition_factor'].min():.3f}")
    print(f"  Max K: {df['condition_factor'].max():.3f}")
    
    return df


def calculate_fish_health_ranges_with_population(df):
    """
    Calculate health ranges using IQR method - includes population
    """
    df = clean_fish_length(df)
    
    # Remove rows where fish metrics are missing
    df_clean = df.dropna(subset=['fish_length', 'fish_weight', 'population'])
    
    print("\n" + "="*70)
    print("FISH HEALTH RANGES (WITH POPULATION)")
    print("="*70)
    print(f"Valid records: {len(df_clean)} / {len(df)}")
    
    FISH_HEALTH_RANGES = {}
    
    # Fish Length (higher is better)
    data = df_clean['fish_length']
    Q1, Q2, Q3 = data.quantile([0.25, 0.50, 0.75])
    IQR = Q3 - Q1
    FISH_HEALTH_RANGES['fish_length'] = {
        'optimal': (Q2, Q3),
        'acceptable': (Q1, Q3 + 0.5*IQR),
        'critical_low': Q1 - 1.5*IQR,
        'critical_high': Q3 + 1.5*IQR,
        'weight': 0.35,
        'direction': 'higher_better'
    }
    print(f"\n✓ Fish Length:")
    print(f"  Q1: {Q1:.2f} | Q2 (median): {Q2:.2f} | Q3: {Q3:.2f}")
    print(f"  Optimal: {Q2:.2f} - {Q3:.2f} cm")
    
    # Fish Weight (higher is better)
    data = df_clean['fish_weight']
    Q1, Q2, Q3 = data.quantile([0.25, 0.50, 0.75])
    IQR = Q3 - Q1
    FISH_HEALTH_RANGES['fish_weight'] = {
        'optimal': (Q2, Q3),
        'acceptable': (Q1, Q3 + 0.5*IQR),
        'critical_low': Q1 - 1.5*IQR,
        'critical_high': Q3 + 1.5*IQR,
        'weight': 0.35,
        'direction': 'higher_better'
    }
    print(f"\n✓ Fish Weight:")
    print(f"  Q1: {Q1:.2f} | Q2 (median): {Q2:.2f} | Q3: {Q3:.2f}")
    print(f"  Optimal: {Q2:.2f} - {Q3:.2f} g")
    
    # Population (lower is better - less crowding)
    data = df_clean['population']
    Q1, Q2, Q3 = data.quantile([0.25, 0.50, 0.75])
    IQR = Q3 - Q1
    FISH_HEALTH_RANGES['population'] = {
        'optimal': (Q1, Q2),  # Lower half is better
        'acceptable': (Q1 - 0.5*IQR, Q3),
        'critical_low': Q1 - 1.5*IQR,
        'critical_high': Q3 + 1.5*IQR,
        'weight': 0.15,
        'direction': 'lower_better'
    }
    print(f"\n✓ Population Density:")
    print(f"  Q1: {Q1:.0f} | Q2 (median): {Q2:.0f} | Q3: {Q3:.0f}")
    print(f"  Optimal: {Q1:.0f} - {Q2:.0f} fish (lower = less crowding)")
    
    # Condition Factor weight
    FISH_HEALTH_RANGES['condition_factor'] = {'weight': 0.15}
    
    print("\n" + "="*70)
    print("WEIGHT DISTRIBUTION:")
    print("  - Fish Length: 35%")
    print("  - Fish Weight: 35%")
    print("  - Population: 15%")
    print("  - Condition Factor: 15%")
    print("="*70)
    
    return FISH_HEALTH_RANGES


def calculate_metric_score(value, param_name, ranges):
    """Calculate health score (0-100) for a single metric"""
    if param_name not in ranges or pd.isna(value):
        return np.nan
    
    r = ranges[param_name]
    opt_low, opt_high = r['optimal']
    acc_low, acc_high = r['acceptable']
    crit_low = r['critical_low']
    crit_high = r['critical_high']
    direction = r.get('direction', 'higher_better')
    
    # Population: lower is better (less overcrowding)
    if direction == 'lower_better':
        if opt_low <= value <= opt_high:
            return 100
        elif value < opt_low:
            return 100 if value >= acc_low else max(50, 75 + 25 * (value - crit_low) / (acc_low - crit_low))
        elif value > opt_high:
            if value <= acc_high:
                return 75 - 25 * (value - opt_high) / (acc_high - opt_high)
            elif value <= crit_high:
                return 25 + 50 * (crit_high - value) / (crit_high - acc_high)
            else:
                return 10  # Severe overcrowding
    
    # Length/Weight: higher is better (good growth)
    else:
        if opt_low <= value <= opt_high:
            return 100
        elif value > opt_high:
            return 100 if value <= acc_high else max(90, 100 - 10 * (value - acc_high) / (crit_high - acc_high))
        elif value < opt_low:
            if value >= acc_low:
                return 75 - 25 * (opt_low - value) / (opt_low - acc_low)
            elif value >= crit_low:
                return 25 + 50 * (value - crit_low) / (acc_low - crit_low)
            else:
                return 10  # Stunted growth
    
    return 50  # Default fallback


def score_condition_factor(k):
    """
    Score Condition Factor (K)
    Scoring:
    - 100: K between 1.0 and 1.5 (optimal)
    - 75-99: K between 0.8-1.0 or 1.5-2.0 (acceptable)
    - 25-74: K between 0.5-0.8 or 2.0-3.0 (poor)
    - 10: K below 0.5 or above 3.0 (critical)
    """
    if pd.isna(k):
        return np.nan
    
    # Optimal range
    if 1.0 <= k <= 1.5:
        return 100
    
    # Acceptable range
    elif (0.8 <= k < 1.0):
        return 75 + 25 * (k - 0.8) / 0.2
    elif (1.5 < k <= 2.0):
        return 75 + 25 * (2.0 - k) / 0.5
    
    # Poor range
    elif (0.5 <= k < 0.8):
        return 25 + 50 * (k - 0.5) / 0.3
    elif (2.0 < k <= 3.0):
        return 25 + 50 * (3.0 - k) / 1.0
    
    # Critical
    else:
        return 10


def add_fish_health_scores_with_population(df):
    print("\n" + "="*70)
    print("CALCULATING FISH HEALTH SCORES (WITH POPULATION)")
    print("="*70)
    
    # Clean and calculate condition factor
    df = clean_fish_length(df)
    df = calculate_condition_factor(df)
    
    # Calculate ranges from data
    FISH_HEALTH_RANGES = calculate_fish_health_ranges_with_population(df)
    
    print("\n" + "="*70)
    print("CALCULATING INDIVIDUAL METRIC SCORES")
    print("="*70)
    
    # Individual metric scores
    df['fish_length_score'] = df['fish_length'].apply(
        lambda x: calculate_metric_score(x, 'fish_length', FISH_HEALTH_RANGES)
    )
    print("✓ Fish length score calculated")
    
    df['fish_weight_score'] = df['fish_weight'].apply(
        lambda x: calculate_metric_score(x, 'fish_weight', FISH_HEALTH_RANGES)
    )
    print("✓ Fish weight score calculated")
    
    df['population_score'] = df['population'].apply(
        lambda x: calculate_metric_score(x, 'population', FISH_HEALTH_RANGES)
    )
    print("✓ Population score calculated")
    
    df['condition_factor_score'] = df['condition_factor'].apply(score_condition_factor)
    print("✓ Condition factor score calculated")
    
    # Overall Fish Health Score (weighted average)
    df['fish_health_score'] = (
        0.35 * df['fish_length_score'].fillna(0) +
        0.35 * df['fish_weight_score'].fillna(0) +
        0.15 * df['population_score'].fillna(0) +
        0.15 * df['condition_factor_score'].fillna(0)
    )
    print("\n✓ Overall fish health score calculated")
    
    # Fish Health Category
    def get_health_category(score):
        if pd.isna(score):
            return 'UNKNOWN'
        if score >= 80:
            return 'EXCELLENT'
        elif score >= 65:
            return 'GOOD'
        elif score >= 50:
            return 'FAIR'
        elif score >= 35:
            return 'POOR'
        else:
            return 'CRITICAL'
    
    df['fish_health_category'] = df['fish_health_score'].apply(get_health_category)
    return df

if __name__ == "__main__":
    
    # Load dataframe WITH population
    df = df_combined.copy()
    df = add_fish_health_scores_with_population(df)
    
    # Save results
    output_file = 'data_with_fish_health_and_population.csv'
    df.to_csv(output_file, index=False)


CALCULATING FISH HEALTH SCORES (WITH POPULATION)

✓ Condition Factor (K) calculated
  Mean K: 0.906
  Min K: 0.474
  Max K: 8.601

FISH HEALTH RANGES (WITH POPULATION)
Valid records: 612529 / 764315

✓ Fish Length:
  Q1: 12.96 | Q2 (median): 17.52 | Q3: 23.42
  Optimal: 17.52 - 23.42 cm

✓ Fish Weight:
  Q1: 18.10 | Q2 (median): 47.20 | Q3: 119.32
  Optimal: 47.20 - 119.32 g

✓ Population Density:
  Q1: 50 | Q2 (median): 50 | Q3: 75
  Optimal: 50 - 50 fish (lower = less crowding)

WEIGHT DISTRIBUTION:
  - Fish Length: 35%
  - Fish Weight: 35%
  - Population: 15%
  - Condition Factor: 15%

CALCULATING INDIVIDUAL METRIC SCORES
✓ Fish length score calculated
✓ Fish weight score calculated
✓ Population score calculated
✓ Condition factor score calculated

✓ Overall fish health score calculated


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764315 entries, 0 to 764314
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   timestamp               764315 non-null  object 
 1   entry_id                764315 non-null  float64
 2   temperature             764315 non-null  float64
 3   turbidity               764315 non-null  float64
 4   dissolved_oxygen        764315 non-null  float64
 5   ph                      764315 non-null  float64
 6   ammonia                 764225 non-null  float64
 7   nitrate                 764315 non-null  float64
 8   population              612530 non-null  float64
 9   fish_length             764314 non-null  float64
 10  fish_weight             764315 non-null  float64
 11  water_safety_score      764315 non-null  float64
 12  safety_category         764315 non-null  object 
 13  source_df               764315 non-null  int64  
 14  condition_factor    